### Подготовка (импорт библиотек, импорт данных базы)

In [541]:
# Импортируем нужные библиотеки
import psycopg # Это Psycopg 3
import pandas

In [542]:
# Корректируем transaction.csv
transaction = pandas.read_csv('transaction.csv', sep=';')
# Удаляем некорректные значения
transaction.dropna(how='any', inplace=True)
transaction=transaction[transaction['customer_id'] != 5034]
transaction.to_csv("transaction_fixed.csv", index=None, sep=';')
transaction

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"
2,3,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,"1793,43","248,82"
3,4,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,"1198,46","381,10"
4,5,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,"1765,3","709,48"
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,24.06.2017,True,Approved,OHM Cycles,Standard,high,medium,"2005,66","1203,40"
19996,19997,41,127,09.11.2017,True,Approved,Solex,Road,medium,medium,"416,98","312,74"
19997,19998,87,2284,14.04.2017,True,Approved,OHM Cycles,Standard,medium,medium,"1636,9","44,71"
19998,19999,6,2764,03.07.2017,False,Approved,OHM Cycles,Standard,high,medium,"227,88","136,73"


In [543]:
# Подключаемся к БД
connection = psycopg.connect(
        host='127.0.0.1',
        user='postgres',
        password='Password123',
        dbname='test'
)

In [544]:
# Создаём таблицы
connection.execute("""
CREATE TABLE customer (
    customer_id SERIAL PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    gender VARCHAR(20),
    DOB DATE,
    job_title VARCHAR(255),
    job_industry_category VARCHAR(100),
    wealth_segment VARCHAR(50),
    deceased_indicator CHAR(1),
    owns_car CHAR(3),
    address VARCHAR(255),
    postcode VARCHAR(10),
    state VARCHAR(100),
    country VARCHAR(100),
    property_valuation INTEGER
);

CREATE TABLE transaction (
    transaction_id SERIAL PRIMARY KEY,
    product_id INTEGER,
    customer_id INTEGER REFERENCES customer(customer_id),
    transaction_date DATE,
    online_order BOOLEAN,
    order_status VARCHAR(50),
    brand VARCHAR(100),
    product_line VARCHAR(100),
    product_class VARCHAR(50),
    product_size VARCHAR(50),
    list_price DECIMAL(10, 2),
    standard_cost DECIMAL(10, 2)
);
                         """)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=127.0.0.1 user=postgres database=test) at 0x1ab1c0f3410>

In [545]:
# Импортируем customer
cursor = connection.cursor()
with open("customer.csv", "r") as f:
    with cursor.copy("COPY customer FROM STDIN WITH (FORMAT CSV, DELIMITER ';', HEADER)") as copy:
        while data := f.read(100):
            copy.write(data)
cursor.close()
# Проверяем
pandas.DataFrame(connection.execute("SELECT * FROM CUSTOMER").fetchall())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,None,Male,1961-10-03,None,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,n/a,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,1975-08-09,VP Product Management,Health,Mass Customer,N,No,57042 Village Green Point,4511,QLD,Australia,6
3996,3997,Blanch,Nisuis,Female,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,3998,Sarene,Woolley,U,None,Assistant Manager,IT,High Net Worth,N,No,8194 Lien Street,4032,QLD,Australia,7
3998,3999,Patrizius,None,Male,1973-10-24,None,Manufacturing,Affluent Customer,N,Yes,320 Acker Drive,2251,NSW,Australia,7


In [546]:
# Импортируем transaction
cursor = connection.cursor()
cursor.execute("""SET DateStyle = 'German, DMY';""")
with open("transaction_fixed.csv", "r") as f:
    with cursor.copy("COPY transaction FROM STDIN WITH (FORMAT CSV, DELIMITER ';', HEADER)") as copy:
        while data := f.read(100):
            copy.write(data.replace(',','.'))
cursor.close()
pandas.DataFrame(connection.execute("SELECT * FROM TRANSACTION").fetchall())

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48
...,...,...,...,...,...,...,...,...,...,...,...,...
19437,19996,51,1018,2017-06-24,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40
19438,19997,41,127,2017-11-09,True,Approved,Solex,Road,medium,medium,416.98,312.74
19439,19998,87,2284,2017-04-14,True,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71
19440,19999,6,2764,2017-07-03,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73


In [547]:
# Откат изменений
#connection.execute("ROLLBACK")

### 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

In [548]:
pandas.DataFrame(connection.execute("""
    SELECT job_industry_category, COUNT(customer_id) as count
    FROM customer
    GROUP BY job_industry_category
    ORDER BY count DESC;
""").fetchall())

,0,1
0,Manufacturing,799
1,Financial Services,774
2,n/a,656
3,Health,602
4,Retail,358
5,Property,267
6,IT,223
7,Entertainment,136
8,Argiculture,113
9,Telecommunications,72


### 2. Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

In [549]:
pandas.DataFrame(connection.execute("""
    SELECT
        TO_CHAR(transaction_date, 'MM')::integer as month,
        job_industry_category,
        SUM(list_price)
    FROM transaction
    JOIN customer USING(customer_id)
    GROUP BY month, job_industry_category
    ORDER BY month, job_industry_category;
""").fetchall())

,0,1,2
0,1,Argiculture,41796.07
1,1,Entertainment,63065.26
2,1,Financial Services,354975.22
3,1,Health,278872.77
4,1,IT,104621.04
...,...,...,...
115,12,Manufacturing,309109.81
116,12,n/a,294322.03
117,12,Property,116191.97
118,12,Retail,148371.60


### 3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

In [550]:
pandas.DataFrame(connection.execute("""
    SELECT COUNT(transaction_id)
    FROM transaction
    JOIN customer USING(customer_id)
    WHERE online_order = TRUE 
        AND job_industry_category = 'IT'
        AND order_status = 'Approved';
""").fetchall())

,0
0,532


### 4. Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.
Примечание (из пачки): необходимо отсортировать результат по убыванию суммы транзакций и количества транзакций

In [551]:
# Используя только GROUP BY
pandas.DataFrame(connection.execute("""
    SELECT customer_id,
        SUM(list_price) as transaction_sum,
        MAX(list_price),
        MIN(list_price),
        COUNT(transaction_id) as transaction_count
    FROM transaction
    JOIN customer USING(customer_id)
    GROUP BY customer_id
    ORDER BY transaction_sum DESC, transaction_count DESC;
""").fetchall())

,0,1,2,3,4
0,2183,19071.32,2005.66,230.91,14
1,1129,18349.27,1992.93,290.62,13
2,941,17898.46,2091.47,1057.51,10
3,2788,17258.94,2083.94,183.86,11
4,1302,17035.83,1977.36,71.16,13
...,...,...,...,...,...
3486,2423,202.62,202.62,202.62,1
3487,3189,200.70,100.35,100.35,2
3488,2274,142.98,71.49,71.49,2
3489,2532,71.49,71.49,71.49,1


In [552]:
# Используя только оконные функции
pandas.DataFrame(connection.execute("""
    SELECT DISTINCT
        customer_id,
        SUM(list_price) OVER (PARTITION BY customer_id) AS transaction_sum,
        MAX(list_price) OVER (PARTITION BY customer_id),
        MIN(list_price) OVER (PARTITION BY customer_id),
        COUNT(transaction_id) OVER (PARTITION BY customer_id) AS transaction_count
    FROM transaction
    JOIN customer USING(customer_id)
    ORDER BY transaction_sum DESC, transaction_count DESC;
""").fetchall())

,0,1,2,3,4
0,2183,19071.32,2005.66,230.91,14
1,1129,18349.27,1992.93,290.62,13
2,941,17898.46,2091.47,1057.51,10
3,2788,17258.94,2083.94,183.86,11
4,1302,17035.83,1977.36,71.16,13
...,...,...,...,...,...
3486,2423,202.62,202.62,202.62,1
3487,3189,200.70,100.35,100.35,2
3488,2274,142.98,71.49,71.49,2
3489,2532,71.49,71.49,71.49,1


__Сравнение__:

Запрос с использованием `group by` имеет более понятный вид, в отличие от запрос с оконными функциями.

Более того, первый запрос выполняется быстрее ([теоретически](https://bertwagner.com/posts/window-functions-vs-group-bys/)).

Однако оконные функции удобно использовать для сложных задач агрегации.

### 5. Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

In [553]:
# Минимальная сумма транзакций
pandas.DataFrame(connection.execute("""
    WITH sum_transaction AS (
        SELECT
            first_name,
            last_name,
            COALESCE(SUM(list_price), 0) AS sum
        FROM customer
        JOIN transaction USING(customer_id)
        GROUP BY (first_name, last_name)
    )
    SELECT *
    FROM sum_transaction
    WHERE sum IN (SELECT MIN(sum) FROM sum_transaction);  
""").fetchall())

,0,1,2
0,Hamlen,Slograve,60.34


In [ ]:
# Максимальная сумма транзакций
pandas.DataFrame(connection.execute("""
    WITH sum_transaction AS (
        SELECT
            first_name,
            last_name,
            COALESCE(SUM(list_price), 0) AS sum
        FROM customer
        JOIN transaction USING(customer_id)
        GROUP BY (first_name, last_name)
    )
    SELECT *
    FROM sum_transaction
    WHERE sum IN (SELECT MAX(sum) FROM sum_transaction);  
""").fetchall())

,0,1,2
0,Jillie,Fyndon,19071.32


### 6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

In [555]:
pandas.DataFrame(connection.execute("""
    SELECT transaction_id
    FROM (
        SELECT transaction_id,
            ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY transaction_date) as number
        FROM customer
        JOIN transaction USING(customer_id)
    )
    WHERE number = 1;
""").fetchall())

,0
0,9785
1,2261
2,10302
3,12441
4,2291
...,...
3486,9769
3487,8276
3488,13469
3489,2794


### 7. Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)

In [556]:
pandas.DataFrame(connection.execute("""
    WITH s1 AS(
        SELECT first_name,
            last_name,
            job_title,
                COALESCE(
                    (transaction_date - LAG(transaction_date)
                        OVER (PARTITION BY customer_id ORDER BY transaction_date)
                    ), 0
                ) AS interval_days
        FROM customer
        JOIN transaction USING(customer_id)
    )
    SELECT *
    FROM s1
    WHERE interval_days IN (SELECT MAX(interval_days) FROM s1);
""").fetchall())

,0,1,2,3
0,Susanetta,None,Legal Assistant,357


In [557]:
# Отключение от БД
connection.close()